In [1]:
# Import BeautifulSoup
from bs4 import BeautifulSoup
from splinter import Browser
import pandas as pd
import time
import requests 
# from flask_pymongo import PyMongo
# !pip install Flask-PyMongo


In [2]:
# Use Chrome driver and pass the string chrome when creating the Browser instance:
def init_browser():
    executable_path = {"executable_path": "chromedriver"}
    return Browser("chrome", **executable_path, headless=False)

In [3]:
# Define a scrape funtion with url as a parameter
def scrape(url):
    browser = init_browser()
    
    # Visit the specified url website
    browser.visit(url)
    
    # Introduce an intentional delay or lag in the scraper to wait 5 second before making the next request
    time.sleep(5)
    
    # Scrape page into Soup
    html = browser.html
    soup = bs(html, "html.parser")

# NASA Mars News

In [4]:

browser = init_browser()
    
url_mars = "https://mars.nasa.gov/news/"
browser.visit(url_mars)
    

time.sleep(2)    

html = browser.html
news_soup = BeautifulSoup(html, "html.parser")

articles = {}


news_title = soup.find('div', class_='content_title').text


news_para = soup.find('div', class_='article_teaser_body').text
    
articles = {
     "news title": news_title,
     "news paragraph": news_para }

print(articles)

{'news title': 'NASA InSight Landing on Mars: Milestones', 'news paragraph': "On Nov. 26, NASA's InSight spacecraft will blaze through the Martian atmosphere and set a lander gently on the surface in less time than it takes to cook a hard-boiled egg."}


# JPL Mars Space Images - Featured Image

In [5]:
url_img = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url_img)

time.sleep(2)

html = browser.html
image_soup = bs(html, "html.parser")

# Get full image for the current featured Mars image and assign it to a variable  
base_url = "https://www.jpl.nasa.gov" 
image = image_soup.find('a', {'id':'full_image','data-fancybox-href': True}).get('data-fancybox-href')
featured_image_url = base_url+image
print(featured_image_url) 


https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA15254_ip.jpg


# Mars Weather

In [6]:
url_tweet = "https://twitter.com/marswxreport?lang=en"
browser.visit(url_tweet)

time.sleep(2)
    
# Scrape page into Soup
html = browser.html
soup_tweet = BeautifulSoup(html, "html.parser")

# Get the news title
mars_weather = soup_tweet.find('p', class_='TweetTextSize TweetTextSize--normal js-tweet-text tweet-text').text

print(mars_weather)

Sol 2234 (2018-11-18), high 2C/35F, low -70C/-93F, pressure at 8.57 hPa, daylight 06:25-18:41


# Mars Facts

In [7]:
url_facts = "http://space-facts.com/mars/"
browser.visit(url_facts)

time.sleep(5)

facts_df = pd.read_html(url_facts)


mars_facts_df = pd.DataFrame(facts_df[0])


mars_facts_df.columns = ['Description','Data']
mars_facts_new = mars_facts_df.set_index("Description")
mars_facts_new

mfact_df = pd.read_html('https://space-facts.com/mars/')[0]
# print(df)
mfact_df.columns=['description', 'value']
mfact_df.set_index('description', inplace=True)
mfact_df

,Data
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [8]:
# Convert the data to a HTML table string  
mars_facts_html = mfact_df.to_html(classes='marsdata')
mars_facts_table = mars_facts_html.replace('\n', ' ')
mars_facts_table

'<table border="1" class="dataframe marsdata">   <thead>     <tr style="text-align: right;">       <th></th>       <th>Data</th>     </tr>     <tr>       <th>Description</th>       <th></th>     </tr>   </thead>   <tbody>     <tr>       <th>Equatorial Diameter:</th>       <td>6,792 km</td>     </tr>     <tr>       <th>Polar Diameter:</th>       <td>6,752 km</td>     </tr>     <tr>       <th>Mass:</th>       <td>6.42 x 10^23 kg (10.7% Earth)</td>     </tr>     <tr>       <th>Moons:</th>       <td>2 (Phobos &amp; Deimos)</td>     </tr>     <tr>       <th>Orbit Distance:</th>       <td>227,943,824 km (1.52 AU)</td>     </tr>     <tr>       <th>Orbit Period:</th>       <td>687 days (1.9 years)</td>     </tr>     <tr>       <th>Surface Temperature:</th>       <td>-153 to 20 °C</td>     </tr>     <tr>       <th>First Record:</th>       <td>2nd millennium BC</td>     </tr>     <tr>       <th>Recorded By:</th>       <td>Egyptian astronomers</td>     </tr>   </tbody> </table>'

# Mars Hemispheres

In [9]:
url_hemisphere = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url_hemisphere)

html = browser.html
soup_hemisphere = bs(html, "html.parser")

img_title = soup_hemisphere.find('div', class_='collapsible results')

hemisphere_image_urls = []

for i in range(len(img_title.find_all("div", class_="item"))):
        time.sleep(2)
        img = browser.find_by_tag('h3')
        img[i].click()
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')
        title = soup.find("h2", class_="title").text
        div = soup.find("div", class_="downloads")
        for li in div:
            link = div.find('a')
        url = link.attrs['href']
        hemisphere_dict = {
                'title' : title,
                'img_url' : url
            }
        hemisphere_image_urls.append(hemisphere_dict)
        browser.back()

print(hemisphere_image_urls)

# Close the browser after scraping
browser.quit()

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]
